In [2]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
base_url = "https://www.basketball-reference.com/leagues/NBA_"
boxscore_url = "https://www.basketball-reference.com/"
# https://www.basketball-reference.com/leagues/NBA_2022_games-november.html
years = range(2000, 2020, 1)
months = ["october", "november", "december", "january", "february", "march", "april", "may", "june", "july", "august", "september"]

In [ ]:
# years = [2022]
# months = ["november"]
index = 0
rows = []
for year in years:
    for month in months:
        try:
            page = requests.get(base_url + str(year) + "_games-" + month + ".html")
            soup = BeautifulSoup(page.content, 'html.parser')
            print(str(year) + " " + month)
        except:
            print("month and year combo don't match")
            pass
        try:
            games = soup.find_all(lambda tag: tag.name == 'td' and tag.get('class') == ['center'] and 'OT' not in tag.text)
        except:
            print("something went wrong with getting the games")
            pass
        for index, game in enumerate(games):
            try:
                boxscore = requests.get(boxscore_url + game.a['href'])
                boxsoup = BeautifulSoup(boxscore.content, 'html.parser')
                teams = boxsoup.find_all(lambda tag: tag.name == 'h2' and '-' in tag.text)
                team1, team2 = teams[0].text.split(' ('), teams[1].text.split(' (')
                team1_name, team2_name = team1[0], team2[0]
                team1, team2 = team1[1].split('-'), team2[1].split('-')
                team1_win, team2_win = team1[0], team2[0]
                team1_loss, team2_loss = team1[1][:-1], team2[1][:-1]
                row = [index, team1_name, team2_name, team1_win, team1_loss, team2_win, team2_loss]

                boxes = boxsoup.find_all(lambda tag: tag.name == 'table' and '-' in tag.caption.text)
                boxes = [boxes[0].tbody, boxes[1].tbody]
                for box in boxes:
                    players = box.find_all('tr')
                    for i in range(5):
                        stats = players[i].find_all('td')
                        row.extend([players[i].th.a.text, stats[0].text, stats[1].text, stats[2].text, 
                                                     stats[4].text, stats[5].text, stats[7].text, stats[8].text, 
                                                      stats[9].text, stats[10].text, stats[11].text, stats[12].text, 
                                                    stats[14].text, stats[15].text, stats[16].text])
                index += 1
                rows.append(row)
            except:
                print("something went wrong with getting the box scores for game " + str(index))
                pass

2000 october
2000 november
2000 december
2000 january
2000 february
2000 march
2000 april
2000 may
2000 june
2000 july
2000 august
2000 september
2001 october
2001 november
2001 december
2001 january
2001 february
2001 march
something went wrong with getting the box scores for game185
2001 april
2001 may
2001 june
2001 july
2001 august
2001 september
2002 october
2002 november
2002 december
2002 january
2002 february
2002 march
2002 april
2002 may


In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df.to_csv('seasons.csv', index = False)